In [59]:
#%matplotlib notebook
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline
import math

In [60]:
def normalization(data):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)
    return data

In [75]:
df = pd.read_csv("new_dtrain.csv", delimiter=",")
zeros = df[df['latitude'] == 0]
df = df[df['latitude'] != 0]
subset = list(df.columns.values)
subset = subset.remove('price')
data = df.drop_duplicates(subset = subset)
#print(len(data[data.price > 1000000000]))
data = data[data.price < 1000000000]
data = data[data.price < 800000000]
data = data[data.price < 400000000]

data.replace(-1, 0, inplace=True)
data.replace(' С ', '0', inplace=True)

data = shuffle(data)
data = data.reset_index(drop=True)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [76]:
# data = log_feature(data)
#choosing data
X = data[['room_number', 'house_type',
       'built_time', 'appartments_floor', 'all_space', 'state', 'bathroom',
       'balcony', 'balcony_glassed', 'door', 'phone', 'ceiling', 'safety',
       'at_the_hotel', 'internet', 'furniture', 'parking', 'building_floors', 'map_complex', 'floor',
          'trngl_first_point', 'trngl_second_point', 'trngl_third_point']]
# X_geo = data[['geocode_lat', 'geocode_long', 'trngl_first_point', 'trngl_second_point', 'trngl_third_point']]
#'latitude','longitude'
y = data['price']
# x_columns = X.columns
# x_index = X.index
# X = normalization(X)
# X = pd.DataFrame(X, index=x_index, columns=x_columns)
# X = pd.concat([X, X_geo], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)

In [77]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors = 5, n_jobs=-1, p=1)
model.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=1,
          weights='uniform')

In [78]:
gbr_model = GradientBoostingRegressor(n_estimators = 100, learning_rate=0.1, 
                                  random_state=7, max_depth=10)
rfr_model = RandomForestRegressor(n_estimators = 300, random_state=7)

In [79]:
gbr_model.fit(X_train, y_train)
rfr_model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=1, oob_score=False, random_state=7,
           verbose=0, warm_start=False)

# Metrics

In [80]:
print(model.score(X_test, y_test))
print(grb_model.score(X_test, y_test))
print(rfr_model.score(X_test, y_test))

0.872940857946
0.96662406681
0.904692474477


In [81]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print("RFR Mean absolute error: ", mean_absolute_error(rfr_model.predict(X_test), y_test))
print("GBR Mean absolute error: ", mean_absolute_error(grb_model.predict(X_test), y_test))
print("KNN Regressor absolute error: ", mean_absolute_error(model.predict(X_test), y_test))

RFR Mean absolute error:  3229900.05527
GBR Mean absolute error:  1731542.30347
KNN Regressor absolute error:  3667246.15305


# Overfitting

In [86]:
print("RFR Mean absolute error: ", mean_absolute_error(rfr_model.predict(X_train), y_train))
print("GBR Mean absolute error: ", mean_absolute_error(grb_model.predict(X_train), y_train))
print("KNN Regressor absolute error: ", mean_absolute_error(model.predict(X_train), y_train))

RFR Mean absolute error:  1214140.39326
GBR Mean absolute error:  1711291.40563
KNN Regressor absolute error:  2831097.65876


In [82]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

In [83]:
def stack_out(data):
    stack = pd.DataFrame()
    stack['knn'] = model.predict(data)
    stack['rfr'] = rfr_model.predict(data)
    stack['gbr'] = gbr_model.predict(data)
    return stack

In [84]:
linear_model.fit(stack_out(X_train), y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [85]:
print("Stacking output(MAE): ", mean_absolute_error(linear_model.predict(stack_out(X_test)), y_test))

Stacking output(MAE):  3364007.60141
